In [49]:
script_text = """
Pia: Willkommen zu einer neuen Folge von "GrünGeschnackt" - heute mit einer gekürzten Version des Green Newsletters aufgrund der Hamburger Pfingstferien. Ich bin Pia Plastikfrei.

Nico: Und ich bin Nico Nachhaltig. Wir führen euch durch die politischen Updates, anstehenden Veranstaltungen und Möglichkeiten zum Engagement in Hamburg.

Pia: Lasst uns gleich loslegen mit den anstehenden Veranstaltungen! Am Sonntag, den 1. Juni 2025 um 13 Uhr, findet eine Radtour zum neuen Quartier Suurheid in Altona statt.

Nico: Genau, alle Fahrradbegeisterten aufgepasst! Die Tour startet am Spielplatz im Rathenaupark und endet gegen 17:30 Uhr wieder dort mit einem leckeren Eis als Abschluss. Es gibt spannende Einblicke und gute Gespräche unterwegs.

Pia: Weitere Termine, die man sich vormerken sollte, sind unter anderem der "Speakers' Corner" am 4. Juni um 19 Uhr zum Thema "Steuern neu denken - Was ist gerecht?" in der Grünen Kreisgeschäftsstelle Eimsbüttel.

Nico: Auch wichtig ist der digitale Informationsabend für Mitglieder am 10. Juni um 18 Uhr, wo diskutiert wird, was der Koalitionsvertrag von SPD und CDU/CSU für grüne Politik bedeutet.

Pia: Und am 25. Juni um 19 Uhr gibt es eine weitere spannende "Speakers' Corner" zum Thema "Das Bedingungslose Grundeinkommen – eine greifbare Utopie?".

Nico: Für alle Fahrradfans startet dann am 13. Juni das Stadtradeln 2025 in Hamburg. Schließ dich dem Team "GRÜNE HH" an und leg Kilometer für mehr Radförderung und Klimaschutz zurück.

Pia: Außerdem könnt ihr noch am 27. Juni um 19 Uhr über die "Rückkehr zur Wehrpflicht?" diskutieren oder am 12. Juli um 10 Uhr an der Landesmitgliederversammlung teilnehmen.

Nico: Das war eine kompakte Übersicht über die anstehenden Termine - und wer weiß, vielleicht sehen wir uns ja bei der einen oder anderen Veranstaltung!

Pia: Genau, bleibt informiert und engagiert euch für eine grünere Zukunft in Hamburg. Wir freuen uns schon auf den nächsten Newsletter mit weiteren spannenden Themen.

Nico: Bleibt dran und bis zum nächsten Mal! Das war's für heute von "GrünGeschnackt". Tschüss!
"""

In [55]:
script_text = """
Willkommen zu einer neuen Folge von "GrünGeschnackt" - heute mit einer gekürzten Version des Green Newsletters aufgrund der Hamburger Pfingstferien. Ich bin Pia Plastikfrei.
Und ich bin Nico Nachhaltig. Wir führen euch durch die politischen Updates, anstehenden Veranstaltungen und Möglichkeiten zum Engagement in Hamburg.
Lasst uns gleich loslegen mit den anstehenden Veranstaltungen! Am Sonntag, den 1. Juni 2025 um 13 Uhr, findet eine Radtour zum neuen Quartier Suurheid in Altona statt.
Genau, alle Fahrradbegeisterten aufgepasst! Die Tour startet am Spielplatz im Rathenaupark und endet gegen 17:30 Uhr wieder dort mit einem leckeren Eis als Abschluss. Es gibt spannende Einblicke und gute Gespräche unterwegs.
Weitere Termine, die man sich vormerken sollte, sind unter anderem der "Speakers' Corner" am 4. Juni um 19 Uhr zum Thema "Steuern neu denken - Was ist gerecht?" in der Grünen Kreisgeschäftsstelle Eimsbüttel.
Auch wichtig ist der digitale Informationsabend für Mitglieder am 10. Juni um 18 Uhr, wo diskutiert wird, was der Koalitionsvertrag von SPD und CDU/CSU für grüne Politik bedeutet.
Und am 25. Juni um 19 Uhr gibt es eine weitere spannende "Speakers' Corner" zum Thema "Das Bedingungslose Grundeinkommen – eine greifbare Utopie?".
Für alle Fahrradfans startet dann am 13. Juni das Stadtradeln 2025 in Hamburg. Schließ dich dem Team "GRÜNE HH" an und leg Kilometer für mehr Radförderung und Klimaschutz zurück.
Außerdem könnt ihr noch am 27. Juni um 19 Uhr über die "Rückkehr zur Wehrpflicht?" diskutieren oder am 12. Juli um 10 Uhr an der Landesmitgliederversammlung teilnehmen.
Das war eine kompakte Übersicht über die anstehenden Termine - und wer weiß, vielleicht sehen wir uns ja bei der einen oder anderen Veranstaltung!
Genau, bleibt informiert und engagiert euch für eine grünere Zukunft in Hamburg. Wir freuen uns schon auf den nächsten Newsletter mit weiteren spannenden Themen.
Bleibt dran und bis zum nächsten Mal! Das war's für heute von "GrünGeschnackt". Tschüss!
"""

In [52]:
import openai
import os
from dotenv import load_dotenv
from pathlib import Path
from pydub import AudioSegment
import re # For parsing speaker lines

# --- Configuration ---
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

if not openai.api_key:
    print("Error: OPENAI_API_KEY environment variable not set.")
    exit()

client = openai.OpenAI()

# --- Voice Mapping ---
# You can choose from: alloy, echo, fable, onyx, nova, shimmer
VOICE_MAPPING = {
    "Pia": "Kore",
    "Nico": "Kore", # Or another available Gemini prebuilt voice if you find one
    "Narrator": "Kore" # Or another
}
VOICE_MAPPING_OPEN_AI = {
    "Pia": "nova",  # Female-sounding
    "Nico": "verse",  # Male-sounding
    "Narrator": "shimmer" # Neutral, can also be used for system messages
}
GEMINI_TTS_MODEL_NAME = "models/tts-1"
USER_GEMINI_TTS_MODEL = "gemini-2.5-flash-preview-tts"

# --- Output and Temporary Files ---
output_filename = Path("dialogue_podcast_gemini.mp3")
temp_audio_dir = Path("temp_audio_segments")


In [47]:
def parse_script(text):
    """Splits the script into speaker and line segments."""
    lines = text.strip().split('\n')
    segments = []
    current_speaker = "Narrator" # Default for lines without explicit speaker

    for line in lines:
        line = line.strip()
        if not line: # Skip empty lines
            continue

        match = re.match(r"^(Pia|Nico):\s*(.*)", line, re.IGNORECASE)
        if match:
            speaker = match.group(1).capitalize() # Normalize to Pia or Nico
            dialogue = match.group(2).strip()
            if dialogue: # Only add if there's actual dialogue
                segments.append({"speaker": speaker, "text": dialogue})
            current_speaker = speaker # Update current speaker for subsequent non-prefixed lines (if any, though not in this script)

    # Refined parsing for the given specific script structure:
    parsed_segments = []
    raw_lines = text.strip().split('\n')
    talker = 0
    for r_line in raw_lines:
        r_line = r_line.strip()
        if not r_line:
            continue

        pia_match = re.match(r"^Pia:\s*(.*)", r_line, re.IGNORECASE)
        nico_match = re.match(r"^Nico:\s*(.*)", r_line, re.IGNORECASE)

        if talker == 0:
            talker = 1
            # dialogue = pia_match.group(1).strip()
            # if dialogue:
            parsed_segments.append({"speaker": "Pia", "text": r_line})
        else:
            talker = 0
            # dialogue = nico_match.group(1).strip()
            # if dialogue:
            parsed_segments.append({"speaker": "Nico", "text": r_line})
        # else:
        #     # Treat as narrator if not Pia or Nico
        #     parsed_segments.append({"speaker": "Narrator", "text": r_line})
    return parsed_segments


In [32]:
from google import genai
from google.genai import types
import wave

load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_TOKEN")
# client = genai.Client(api_key=GEMINI_API_KEY)

In [33]:
def generate_speech_segment_openai(text_input, voice_name, segment_filepath):
    """Generates speech for a text segment and saves it."""
    try:
        print(f"🔊 Generating speech for '{text_input[:30]}...' with voice '{voice_name}'")
        with client.audio.speech.with_streaming_response.create(
            model="gpt-4o-mini-tts",
            voice=voice_name,
            input=text_input,
            # response_format="mp3" # mp3 is default
        ) as response:
            response.stream_to_file(segment_filepath)
        print(f"   Segment saved: {segment_filepath}")
        return True
    except Exception as e:
        print(f"❌ Error generating speech for '{text_input[:30]}...': {e}")
        if "401" in str(e):
             print("   This might be an issue with your OpenAI API key or organization setup for TTS.")
        return False

In [29]:
def wave_file(filename, pcm, channels=1, rate=24000, sample_width=2):
    print("Saving file ...")
    with wave.open(filename, "wb") as wf:
      wf.setnchannels(channels)
      wf.setsampwidth(sample_width)
      wf.setframerate(rate)
      wf.writeframes(pcm)


def generate_speech_segment(text_input, voice_name, segment_filepath_wav):
    print("Creating text to speech segment...")
    print(f"FilePath: {segment_filepath_wav}")
    print(f"Voice: {voice_name}")
    print(f"Text: {text_input}")
    GEMINI_API_KEY = os.getenv("GEMINI_API_TOKEN")
    client = genai.Client(api_key=GEMINI_API_KEY)

    response = client.models.generate_content(
       model="gemini-2.5-flash-preview-tts",
       contents=f"Lies bitte folgenden Text vor, als wärest du Teil eines Podcasts: {text_input}",
       config=types.GenerateContentConfig(
          response_modalities=["AUDIO"],
          speech_config=types.SpeechConfig(
             voice_config=types.VoiceConfig(
                prebuilt_voice_config=types.PrebuiltVoiceConfig(
                   voice_name=voice_name,
                )
             )
          ),
       )
    )

    data = response.candidates[0].content.parts[0].inline_data.data

    print("Saving file ...")
    with wave.open(str(segment_filepath_wav), "wb") as wf:
      wf.setnchannels(1)
      wf.setsampwidth(2)
      wf.setframerate(24000)
      wf.writeframes(data)


In [56]:
# Create temporary directory if it doesn't exist
temp_audio_dir.mkdir(parents=True, exist_ok=True)

parsed_dialogue = parse_script(script_text)

if not parsed_dialogue:
    print("No dialogue segments found. Exiting.")
    exit("No parsed dialogue")

segment_files = []
combined_audio = AudioSegment.empty()

print(f"\nProcessing {len(parsed_dialogue)} dialogue segments...")

for i, segment_data in enumerate(parsed_dialogue):
    speaker = segment_data["speaker"]
    text = segment_data["text"]
    voice = VOICE_MAPPING_OPEN_AI.get(speaker, VOICE_MAPPING["Narrator"]) # Default to Narrator voice if speaker not in map

    segment_filename = temp_audio_dir / f"segment_{i:03d}_{speaker.lower()}.mp3"

    if generate_speech_segment_openai(text, voice, segment_filename):
        try:
            audio_segment = AudioSegment.from_mp3(segment_filename)
            combined_audio += audio_segment
            # Add a small silence between segments, except for the last one
            if i < len(parsed_dialogue) - 1:
                if speaker == "Narrator" and (parsed_dialogue[i+1]["speaker"] == "Pia" or parsed_dialogue[i+1]["speaker"] == "Nico"):
                    combined_audio += AudioSegment.silent(duration=700) # Longer pause before character speaks
                elif (speaker == "Pia" or speaker == "Nico") and parsed_dialogue[i+1]["speaker"] == "Narrator":
                    combined_audio += AudioSegment.silent(duration=700) # Longer pause before narrator speaks
                elif speaker != parsed_dialogue[i+1]["speaker"]: # Different speaker
                    combined_audio += AudioSegment.silent(duration=500) # Pause between different speakers
                else: # Same speaker (though unlikely with current parsing)
                    combined_audio += AudioSegment.silent(duration=200)
            segment_files.append(segment_filename) # Keep track for cleanup
        except Exception as e:
            print(f"❌ Error loading or concatenating audio segment {segment_filename}: {e}")
    else:
        print(f"Skipping segment due to TTS generation error: {text[:30]}...")


if len(segment_files) > 0 and len(combined_audio) > 0 :
    try:
        print(f"\n🎵 Exporting combined audio to {output_filename}...")
        combined_audio.export(output_filename, format="mp3")
        print(f"✅ Successfully created {output_filename}")
    except Exception as e:
        print(f"❌ Error exporting combined audio: {e}")
else:
    print("No audio segments were successfully generated or combined. Output file not created.")

# --- Cleanup ---
# try:
#     print(f"\n🧹 Cleaning up temporary directory: {temp_audio_dir}")
#     shutil.rmtree(temp_audio_dir)
#     print("   Temporary directory cleaned up.")
# except Exception as e:
#     print(f"   Error cleaning up temporary directory: {e}")
print(f"\nℹ️ Temporary audio segments are in: {temp_audio_dir.resolve()}")
print("   You may want to delete this directory manually afte   r checking the segments.")


Processing 12 dialogue segments...
🔊 Generating speech for 'Willkommen zu einer neuen Folg...' with voice 'nova'
   Segment saved: temp_audio_segments/segment_000_pia.mp3
❌ Error loading or concatenating audio segment temp_audio_segments/segment_000_pia.mp3: [Errno 2] No such file or directory: 'ffprobe'
🔊 Generating speech for 'Und ich bin Nico Nachhaltig. W...' with voice 'fable'


/home/philipp/anaconda3/lib/python3.12/site-packages/pydub/utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


   Segment saved: temp_audio_segments/segment_001_nico.mp3
❌ Error loading or concatenating audio segment temp_audio_segments/segment_001_nico.mp3: [Errno 2] No such file or directory: 'ffprobe'
🔊 Generating speech for 'Lasst uns gleich loslegen mit ...' with voice 'nova'


/home/philipp/anaconda3/lib/python3.12/site-packages/pydub/utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


   Segment saved: temp_audio_segments/segment_002_pia.mp3
❌ Error loading or concatenating audio segment temp_audio_segments/segment_002_pia.mp3: [Errno 2] No such file or directory: 'ffprobe'
🔊 Generating speech for 'Genau, alle Fahrradbegeisterte...' with voice 'fable'


...

/home/philipp/anaconda3/lib/python3.12/site-packages/pydub/utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


   Segment saved: temp_audio_segments/segment_010_pia.mp3
❌ Error loading or concatenating audio segment temp_audio_segments/segment_010_pia.mp3: [Errno 2] No such file or directory: 'ffprobe'
🔊 Generating speech for 'Bleibt dran und bis zum nächst...' with voice 'fable'


/home/philipp/anaconda3/lib/python3.12/site-packages/pydub/utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


   Segment saved: temp_audio_segments/segment_011_nico.mp3
❌ Error loading or concatenating audio segment temp_audio_segments/segment_011_nico.mp3: [Errno 2] No such file or directory: 'ffprobe'
No audio segments were successfully generated or combined. Output file not created.

ℹ️ Temporary audio segments are in: /home/philipp/projects/gruene/podcast/temp_audio_segments
   You may want to delete this directory manually afte   r checking the segments.


/home/philipp/anaconda3/lib/python3.12/site-packages/pydub/utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)
